In [1]:
import hashlib
import secrets

## Naive Lamport Signature

This is a basic implementation of the original Lamport Signatures paper with uncompressed public and private keys. The private key is a 256 bit random number and the public key is a 256 bit hash of the private key. The signature is a 256 bit hash of the message and the private key.

### Example: 3 Bit Keypair Generation 

![3 Bit Keypair Generation](./images/3_bit_keypair.png)

### Example: 3 Bit Message Signing

![3 Bit Message Signing](./images/3_bit_signing.png)

### Example: 3 Bit Signature Verification

![3 Bit Signature Verification](./images/3_bit_verification.png)

In [2]:
####### CONSTANTS #######

# Provides roughly 128 bits of security (see Grover's algorithm)
security_parameter = 256

# Max message length that the private key can sign
max_message_length = 256

### Helper Functions

In [3]:
def hash_key_element(key_element: int) -> bytes:
    '''Hashes a single key element, producing a 256-bit digest'''

    bytes_length = security_parameter // 8
    return hashlib.sha3_256(key_element.to_bytes(
        length=bytes_length,
        byteorder="big",
    )).digest()


def hash_message(message: str) -> bytes:
    '''Hashes a message using SHA3-256'''

    return hashlib.sha3_256(message.encode()).digest()


def choose_key_elements(message_hash: bytes, key: list) -> list:
    '''Chooses the key elements depending on input message hash bits (either 1 or 0)'''
    
    output = []
    for i in range(max_message_length // 8):
        for j in range(8):
            bit_index = i * 8 + j
            bit_mask = 0b10000000 >> j

            # Determine which key to use based on the message hash bit
            key_bit = int(message_hash[i] & bit_mask != 0) # either 0 or 1
            key_index = 2*bit_index + key_bit
            
            # Add the key elements to output
            output.append(key[key_index])

    return output

### Naive Lamport Signature Functions

The functions below provide the interface to generate a keypair and sign a message.

### Key Size Comparison

| Scheme | Private Key | Public Key | Total Size |
| --- | --- | --- | --- |
| Lamport: Naive  | $32 * 256 = 8192$ bytes | $32 * 256 = 8192$ bytes | $16384$ bytes |

### Signature Size Comparison

| Scheme | Signature |
| --- | --- |
| Lamport: Naive  | $32 * 256 = 8192$ bytes |


In [4]:
PrivateKey = list[int]
PublicKey = list[bytes]
Signature = list[int]

def generate_private_key() -> PrivateKey:
    '''Generates a private Lamport key'''

    # Generate the private key from random bits
    return [secrets.randbits(security_parameter) for _ in range(2 * max_message_length)]


def generate_public_key(private_key: PrivateKey) -> PublicKey:
    '''Generates a public Lamport key from a private key'''

    # Generate public key by hashing each private key element
    return list(map(hash_key_element, private_key))


def generate_keypair() -> tuple[PublicKey, PrivateKey]:
    '''Generates a public and private Lamport keypair'''

    # Generate the private key from random bits
    private_key = generate_private_key()

    # Generate public key by hashing each private key element
    public_key = generate_public_key(private_key)

    return (public_key, private_key)


def sign_message(message: str, private_key: PublicKey) -> Signature:
    '''Signs a message with a Lamport private key'''

    # Hash the message and then commit to that hash using the private key
    message_hash = hash_message(message)

    # Choose private key elements depending on the bits of the message hash
    return choose_key_elements(message_hash, private_key)


def verify_signature(message: str, public_key: PublicKey, signature: Signature) -> bool:
    '''Verifies a Lamport signature'''
    
    # Hash the message and then check that the hash of the signature matches the public key
    message_hash = hash_message(message)

    chosen_signature_hashes = choose_key_elements(message_hash, public_key)
    derived_signature_hashes = list(map(hash_key_element, signature))

    return chosen_signature_hashes == derived_signature_hashes

In [5]:
signed_string = "Hello, world!"
public_key, private_key = generate_keypair()
signature = sign_message(signed_string, private_key)
if verify_signature(signed_string, public_key, signature):
    print("Signature is correct!")
else:
    print("Signature is incorrect!")

Signature is correct!


In [6]:
# Try and forge a signature by using the same private key for a different message
random_signature = [secrets.randbits(security_parameter) for _ in range(2 * max_message_length)]
if verify_signature(signed_string, public_key, random_signature):
    print("Forgery is correct!")
else:
    print("Forgery is incorrect!")

Forgery is incorrect!
